In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import make_blobs
from collections import Counter
from path import Path
import hvplot.pandas
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
import mpu
from uszipcode import SearchEngine

ImportError: cannot import name 'MultiChoice' from 'bokeh.models.widgets' (C:\Users\Natalia\anaconda3\lib\site-packages\bokeh\models\widgets\__init__.py)

## import data - pitching, batting, teams acronyms, schedules


In [ ]:
csv_path = Path("../Data Prep/Jack/Resources/2018_pitching.csv")
df_pitch = pd.read_csv(csv_path)
df_pitch.tail()

In [ ]:
csv_path = Path("../Data Prep/Jack/Resources/batting_2018.csv")
df_bat = pd.read_csv(csv_path)
df_bat.tail(10)

In [ ]:
csv_path = Path("../Data Prep/Jack/Resources/2018_astros_sced.csv")
df_sche = pd.read_csv(csv_path)
df_sche.head()

In [ ]:
df_result = pd.read_csv(Path("random.txt"))
df_home_away = pd.read_csv(Path("2018wl.txt"))
df_home_away.head()

In [ ]:
team_acry = pd.read_csv(Path("team_acry.csv"))
team_acry.set_index("Team(Bat)", inplace = True)
team_acry.head()

In [ ]:
def clean_schedule_data(df, df1):
    
    # pulling only the columns we want from the pitching log dataframe
    df = df[["Game#", "Opp", "Result", "Starting Pitcher", "Opposing Pitcher"]]
    df.set_index("Game#", inplace = True)
    
    # pulling only the columns we want from the schedule log dataframe
    df1 = df1[["Hm/Aw", "Game#"]]
    df1.set_index("Game#", inplace = True)
    
    # combining into one dataframe to work with
    return_df = pd.merge(df, df1, left_index=True, right_index=True)
    return_df.dropna(inplace=True)
    
    # changing results from W/L to 1/0
    true_w = return_df["Result"].str.contains('W', na=True)
    binary_result = []
    for row in return_df.index:
        if true_w[row]:
            binary_result.append(1)
        else:
            binary_result.append(0)
        
    return_df["Result"] = binary_result
    
    
    # getting the distance between thw two clubs
    Home = []
    Away = []
    
    x = len(return_df)
    return_df.reset_index(inplace = True)
    return_df.drop(columns = "Game#", inplace = True)
    
    for number in return_df.index:
        if return_df.loc[number]["Hm/Aw"] == "Away":
            Home.append(return_df.loc[number]["Opp"])
            Away.append("HOU")
        else:
            Home.append("HOU")
            Away.append(return_df.loc[number]["Opp"])
            
    return_df["Home"] = Home
    return_df["Away"] = Away
        
    return_df["Distance"] = [get_home_away_distance_in_miles(home, away) for home, away in zip(return_df.Home, return_df.Away)]

    return_df['Distance'] = [distance * -1 if stadium == 'Home' else distance for stadium, 
                             distance in zip(return_df['Hm/Aw'], return_df['Distance'])]
    
    
    # setting index and dropping off columns we don't need anymore
    return_df.set_index("Opp", inplace = True)
    
    return_df.drop(columns = ["Hm/Aw", "Home", "Away"], inplace = True)
    
    return return_df



## Cleaning imported data/calculations for distance between stadiums for playing teams


In [ ]:
def clean_batting_data(df1):
    
    # dropping last two rows as they are average and total
    df1.drop(index = [30, 31], inplace = True)
    
    # setting index to team name
    df1.set_index("Tm", inplace = True)
    
    # changing team names to match
    new_team_name = []

    for team in df1.index:
        if team in team_acry.index:
            new_team_name.append(team_acry.loc[team]["Team(New)"])
            
    df1.index = new_team_name
            
    # setting index to team name
#     df1.set_index("Tm", inplace = True)
    
    # pulling only the metrics we want
    df1 = df1[["BatAge", "R/G", "PA", "AB", "H", "2B", "3B", "HR", "BB", "SO", "BA", "OBP", "SLG", "OPS", "TB", "HBP", "LOB"]]
    
    # returning the dataframe
    return df1

In [ ]:
def clean_pitching_data(df):
    
    # need to get rid of extra space in pitcher names
    random_number = list(range(len(df)))
    list_of_pitchers = []
    for number in random_number:
        string = df["Name"][number] 
        string = string.replace(u'\xa0', u' ')
        list_of_pitchers.append(string)
    df["Pitcher"] = list_of_pitchers
    
    # setting index to pitchers
    df.set_index("Pitcher", inplace = True)
    
    # pulling only the metrics we want
    df = df[["Tm", "ERA", "CG", "IP", "ERA+", "FIP", "WHIP", "H9", "HR9", "BB9", "SO9", "SO/W"]]
    
    # returning the dataframe
    return df

In [ ]:
import pandas as pd
import mpu
from uszipcode import SearchEngine

def get_distance_in_miles(home_team_zip, away_team_zip):        
    
    #for extensive list of zipcodes, set simple_zipcode=False
    search = SearchEngine(simple_zipcode=True)
    zip1 = search.by_zipcode(home_team_zip)
    zip2 = search.by_zipcode(away_team_zip)

    return round(mpu.haversine_distance((zip1.lat, zip1.lng), (zip2.lat, zip2.lng)), 2)

def get_home_away_distance_in_miles(home_team_code, away_team_code):
    
    teams = [
        { 'team': 'ARI', 'name': 'Arizona Diamondbacks', 'address': '401 E Jefferson St, Phoenix, AZ 85004' },
        { 'team': 'ATL', 'name': 'Atlanta Braves',  'address': '755 Battery Ave SE, Atlanta, GA 30339' },
        { 'team': 'BAL', 'name': 'Baltimore Orioles', 'address': '333 W Camden St, Baltimore, MD 21201' },
        { 'team': 'BOS', 'name': 'Boston Red Sox', 'address': '4 Jersey St, Boston, MA 02215' },
        { 'team': 'CHA', 'name': 'Chicago White Sox', 'address': '333 W 35th St, Chicago, IL 60616'},
        { 'team': 'CHN', 'name': 'Chicago Cubs', 'address': '1060 W Addison St, Chicago, IL 60613' },
        { 'team': 'CIN', 'name': 'Cincinnati Reds', 'address': '100 Joe Nuxhall Way, Cincinnati, OH 45202' }, 
        { 'team': 'CLE', 'name': 'Cleveland Indians', 'address': '2401 Ontario St, Cleveland, OH 44115' },
        { 'team': 'COL', 'name': 'Colorado Rockies', 'address': '2001 Blake St, Denver, CO 80205' },
        { 'team': 'DET', 'name': 'Detroit Tigers', 'address': '2100 Woodward Ave, Detroit, MI 48201' },
        { 'team': 'HOU', 'name': 'Houston Astros', 'address': '501 Crawford St, Houston, TX 77002' },
        { 'team': 'KCA', 'name': 'Kansas City Royals', 'address': '1 Royal Way, Kansas City, MO 64129' },
        { 'team': 'ANA', 'name': 'Los Angeles Angels', 'address': '2000 E Gene Autry Way, Anaheim, CA 92806' },
        { 'team': 'LAN', 'name': 'Los Angeles Dodgers', 'address': '1000 Vin Scully Ave, Los Angeles, CA 90012' },
        { 'team': 'MIA', 'name': 'Miami Marlins', 'address': '501 Marlins Way, Miami, FL 33125' },
        { 'team': 'MIL', 'name': 'Milwaukee Brewers', 'address': '1 Brewers Way, Milwaukee, WI 53214' },
        { 'team': 'MIN', 'name': 'Minnesota Twins', 'address': '1 Twins Way, Minneapolis, MN 55403' },
        { 'team': 'NYA', 'name': 'New York Yankees', 'address': '1 E 161 St, The Bronx, NY 10451' },
        { 'team': 'NYN', 'name': 'New York Mets', 'address': '41 Seaver Way, Queens, NY 11368' },
        { 'team': 'OAK', 'name': 'Oakland Athletics', 'address': '7000 Coliseum Way, Oakland, CA 94621' },
        { 'team': 'PHI', 'name': 'Philadelphia Phillies', 'address': '1 Citizens Bank Way, Philadelphia, PA 19148' },
        { 'team': 'PIT', 'name': 'Pittsburgh Pirates', 'address': '115 Federal St, Pittsburgh, PA 15212' },
        { 'team': 'SDN', 'name': 'San Diego Padres', 'address': '100 Park Blvd, San Diego, CA 92101' },
        { 'team': 'SFN', 'name': 'San Francisco Giants', 'address': '24 Willie Mays Plaza, San Francisco, CA 94107' },
        { 'team': 'SEA', 'name': 'Seattle Mariners', 'address': '1250 1st Ave S, Seattle, WA 98134' },
        { 'team': 'SLN', 'name': 'St. Louis Cardinals', 'address': '700 Clark Ave, St. Louis, MO 63102' },
        { 'team': 'TBA', 'name': 'Tampa Bay Rays', 'address': '1 Tropicana Dr., St. Petersburg, FL 33705' },
        { 'team': 'TEX', 'name': 'Texas Rangers', 'address': '734 Stadium Dr, Arlington, TX 76011' },
        { 'team': 'TOR', 'name': 'Toronto Blue Jays', 'address': '1 Blue Jays Way, Toronto, ON 14305' },
        { 'team': 'WAS', 'name': 'Washington Nationals', 'address': '1500 S Capitol St SE, Washington, DC 20003' }
    ]

    teams_zip_df = pd.DataFrame.from_dict(teams)

    teams_zip_df['zip'] = teams_zip_df['address'].str.split(',', expand=True)[2].str.split(' ', expand=True)[2]
    teams_zip_df.sort_values("team", inplace=True) 
    teams_zip_df.set_index('team', inplace=True)    
        
    return get_distance_in_miles(
        home_team_zip = teams_zip_df.loc[home_team_code]['zip'], 
        away_team_zip = teams_zip_df.loc[away_team_code]['zip'])

In [ ]:
import pandas as pd
import mpu
from uszipcode import SearchEngine

def get_distance_in_miles(home_team_zip, away_team_zip):        
    
    #for extensive list of zipcodes, set simple_zipcode=False
    search = SearchEngine(simple_zipcode=True)
    zip1 = search.by_zipcode(home_team_zip)
    zip2 = search.by_zipcode(away_team_zip)

    return round(mpu.haversine_distance((zip1.lat, zip1.lng), (zip2.lat, zip2.lng)), 2)

def get_home_away_distance_in_miles(home_team_code, away_team_code):
    
    teams = [
        { 'team': 'ARI', 'name': 'Arizona Diamondbacks', 'address': '401 E Jefferson St, Phoenix, AZ 85004' },
        { 'team': 'ATL', 'name': 'Atlanta Braves',  'address': '755 Battery Ave SE, Atlanta, GA 30339' },
        { 'team': 'BAL', 'name': 'Baltimore Orioles', 'address': '333 W Camden St, Baltimore, MD 21201' },
        { 'team': 'BOS', 'name': 'Boston Red Sox', 'address': '4 Jersey St, Boston, MA 02215' },
        { 'team': 'CHA', 'name': 'Chicago White Sox', 'address': '333 W 35th St, Chicago, IL 60616'},
        { 'team': 'CHN', 'name': 'Chicago Cubs', 'address': '1060 W Addison St, Chicago, IL 60613' },
        { 'team': 'CIN', 'name': 'Cincinnati Reds', 'address': '100 Joe Nuxhall Way, Cincinnati, OH 45202' }, 
        { 'team': 'CLE', 'name': 'Cleveland Indians', 'address': '2401 Ontario St, Cleveland, OH 44115' },
        { 'team': 'COL', 'name': 'Colorado Rockies', 'address': '2001 Blake St, Denver, CO 80205' },
        { 'team': 'DET', 'name': 'Detroit Tigers', 'address': '2100 Woodward Ave, Detroit, MI 48201' },
        { 'team': 'HOU', 'name': 'Houston Astros', 'address': '501 Crawford St, Houston, TX 77002' },
        { 'team': 'KCA', 'name': 'Kansas City Royals', 'address': '1 Royal Way, Kansas City, MO 64129' },
        { 'team': 'ANA', 'name': 'Los Angeles Angels', 'address': '2000 E Gene Autry Way, Anaheim, CA 92806' },
        { 'team': 'LAN', 'name': 'Los Angeles Dodgers', 'address': '1000 Vin Scully Ave, Los Angeles, CA 90012' },
        { 'team': 'MIA', 'name': 'Miami Marlins', 'address': '501 Marlins Way, Miami, FL 33125' },
        { 'team': 'MIL', 'name': 'Milwaukee Brewers', 'address': '1 Brewers Way, Milwaukee, WI 53214' },
        { 'team': 'MIN', 'name': 'Minnesota Twins', 'address': '1 Twins Way, Minneapolis, MN 55403' },
        { 'team': 'NYA', 'name': 'New York Yankees', 'address': '1 E 161 St, The Bronx, NY 10451' },
        { 'team': 'NYN', 'name': 'New York Mets', 'address': '41 Seaver Way, Queens, NY 11368' },
        { 'team': 'OAK', 'name': 'Oakland Athletics', 'address': '7000 Coliseum Way, Oakland, CA 94621' },
        { 'team': 'PHI', 'name': 'Philadelphia Phillies', 'address': '1 Citizens Bank Way, Philadelphia, PA 19148' },
        { 'team': 'PIT', 'name': 'Pittsburgh Pirates', 'address': '115 Federal St, Pittsburgh, PA 15212' },
        { 'team': 'SDN', 'name': 'San Diego Padres', 'address': '100 Park Blvd, San Diego, CA 92101' },
        { 'team': 'SFN', 'name': 'San Francisco Giants', 'address': '24 Willie Mays Plaza, San Francisco, CA 94107' },
        { 'team': 'SEA', 'name': 'Seattle Mariners', 'address': '1250 1st Ave S, Seattle, WA 98134' },
        { 'team': 'SLN', 'name': 'St. Louis Cardinals', 'address': '700 Clark Ave, St. Louis, MO 63102' },
        { 'team': 'TBA', 'name': 'Tampa Bay Rays', 'address': '1 Tropicana Dr., St. Petersburg, FL 33705' },
        { 'team': 'TEX', 'name': 'Texas Rangers', 'address': '734 Stadium Dr, Arlington, TX 76011' },
        { 'team': 'TOR', 'name': 'Toronto Blue Jays', 'address': '1 Blue Jays Way, Toronto, ON 14305' },
        { 'team': 'WAS', 'name': 'Washington Nationals', 'address': '1500 S Capitol St SE, Washington, DC 20003' }
    ]

    teams_zip_df = pd.DataFrame.from_dict(teams)

    teams_zip_df['zip'] = teams_zip_df['address'].str.split(',', expand=True)[2].str.split(' ', expand=True)[2]
    teams_zip_df.sort_values("team", inplace=True) 
    teams_zip_df.set_index('team', inplace=True)    
        
    return get_distance_in_miles(
        home_team_zip = teams_zip_df.loc[home_team_code]['zip'], 
        away_team_zip = teams_zip_df.loc[away_team_code]['zip'])

## Creating function for win/loss

In [ ]:
def determine_who_wins(starting_pitcher, opposing_pitcher, home_or_away):
    
    
    # gathering pitching data from two inputs in the function
    opp_pitch_class = []
    opp_pitch_class_2 = []
    opp_pitch_class_3 = []
    opp_pitch_class_4 = []
    opp_pitch_class_5 = []
    opp_pitch_class_6 = []
    opp_pitch_class_7 = []
    opp_pitch_class_8 = []
    opp_pitch_class_9 = []
    opp_pitch_class_10 = []
    opp_pitch_class_11 = []
    
    if opposing_pitcher in df_pitch.index:
        opp_pitch_class.append(df_pitch.loc[opposing_pitcher]["ERA"])
        opp_pitch_class_2.append(df_pitch.loc[opposing_pitcher]["CG"])
        opp_pitch_class_3.append(df_pitch.loc[opposing_pitcher]["IP"])
        opp_pitch_class_4.append(df_pitch.loc[opposing_pitcher]["ERA+"])
        opp_pitch_class_5.append(df_pitch.loc[opposing_pitcher]["FIP"])
        opp_pitch_class_6.append(df_pitch.loc[opposing_pitcher]["WHIP"])
        opp_pitch_class_7.append(df_pitch.loc[opposing_pitcher]["H9"])
        opp_pitch_class_8.append(df_pitch.loc[opposing_pitcher]["HR9"])
        opp_pitch_class_9.append(df_pitch.loc[opposing_pitcher]["BB9"])
        opp_pitch_class_10.append(df_pitch.loc[opposing_pitcher]["SO9"])
        opp_pitch_class_11.append(df_pitch.loc[opposing_pitcher]["SO/W"])
    else:
        Print("Opposing Pitcher Not Found")

    start_pitch_class = []
    start_pitch_class_2 = []
    start_pitch_class_3 = []
    start_pitch_class_4 = []
    start_pitch_class_5 = []
    start_pitch_class_6 = []
    start_pitch_class_7 = []
    start_pitch_class_8 = []
    start_pitch_class_9 = []
    start_pitch_class_10 = []
    start_pitch_class_11 = []
       
    if starting_pitcher in df_pitch.index:
        start_pitch_class.append(df_pitch.loc[starting_pitcher]["ERA"])
        start_pitch_class_2.append(df_pitch.loc[starting_pitcher]["CG"])
        start_pitch_class_3.append(df_pitch.loc[starting_pitcher]["IP"])
        start_pitch_class_4.append(df_pitch.loc[starting_pitcher]["ERA+"])
        start_pitch_class_5.append(df_pitch.loc[starting_pitcher]["FIP"])
        start_pitch_class_6.append(df_pitch.loc[starting_pitcher]["WHIP"])
        start_pitch_class_7.append(df_pitch.loc[starting_pitcher]["H9"])
        start_pitch_class_8.append(df_pitch.loc[starting_pitcher]["HR9"])
        start_pitch_class_9.append(df_pitch.loc[starting_pitcher]["BB9"])
        start_pitch_class_10.append(df_pitch.loc[starting_pitcher]["SO9"])
        start_pitch_class_11.append(df_pitch.loc[starting_pitcher]["SO/W"])
    else:
        Print("Starting Pitcher Not Found")
    
        
        
    # gathering batting metrics from pitchers' team
    if opposing_pitcher in df_pitch.index:
        opp_team_name = df_pitch.loc[opposing_pitcher]["Tm"]
    
    if opp_team_name in df_bat.index:
        opp_team_metrics = df_bat.loc[opp_team_name]
        
    opp_team = pd.DataFrame(opp_team_metrics)
    opp_team = opp_team.transpose()
    opp_team["key"] = 1

    if starting_pitcher in df_pitch.index:
        team_name = df_pitch.loc[opposing_pitcher]["Tm"]
    
    if team_name in df_bat.index:
        team = df_bat.loc[team_name]
        
    team = pd.DataFrame(team)
    team = team.transpose()
    team["key"] = 1    
    
    
    # gathering distance between teams
    distance = get_home_away_distance_in_miles(team_name, opp_team_name)
    
    if home_or_away == "Home":
        team["Distance"] = distance
    else:
        team["Distance"] = (distance * -1)
    
        
    # creating a dataframe to combine all the pitching metrics    
    pitching_metrics = pd.DataFrame(
    {     
        "ERA-Starting" : start_pitch_class,
        "ERA-Opposing" : opp_pitch_class,
        "CG-Starting" : start_pitch_class_2,
        "CG-Opposing" : opp_pitch_class_2,
        "IP-Starting" : start_pitch_class_3,
        "IP-Opposing" : opp_pitch_class_3,
        "ERA+-Starting" : start_pitch_class_4,
        "ERA+-Opposing" : opp_pitch_class_4,
        "FIP-Starting" : start_pitch_class_5,
        "FIP-Opposing" : opp_pitch_class_5,
        "WHIP-Starting" : start_pitch_class_6,
        "WHIP-Opposing" : opp_pitch_class_6,
        "H9-Starting" : start_pitch_class_7,
        "H9-Opposing" : opp_pitch_class_7,
        "HR9-Starting" : start_pitch_class_8,
        "HR9-Opposing" : opp_pitch_class_8,
        "BB9-Starting" : start_pitch_class_9,
        "BB9-Opposing" : opp_pitch_class_9,
        "SO9-Starting" : start_pitch_class_10,
        "SO9-Opposing" : opp_pitch_class_10,
        "SO/W-Starting" : start_pitch_class_11,
        "SO/W-Opposing" : opp_pitch_class_10,
        "key" : 1
    })
    
    
    # creating test data to run through model
    batting_metrics = pd.merge(team, opp_team, on='key')
    test_data = pd.merge(batting_metrics, pitching_metrics, on="key")
    test_data.drop(["key"], axis=1, inplace = True)
    
    # testing to see what the results look like before running a model
    # it works, I can comment this out
    # result = test_data
    
    # running our model
    result = model.predict(test_data)
    
    #printing out our prediction
    return result

## importing files for win/loss


In [ ]:
df_2019 = pd.read_csv(Path("random.txt"))
df_2019 = df_2019[["Opp", "Result", "Starting Pitcher", "Opposing Pitcher", "Game Date"]]

true_w = df_2019["Result"].str.contains('W', na=True)
count = 0
Result = []
for row in df_2019.index:
    if true_w[row]:
        Result.append(1)
    else:
        Result.append(0)
        
        
        
true_w = df_2019["Result"].str.contains('W', na=True)

df_2019["Result"] = Result
df_2019 = df_2019.set_index("Opp")

df_2019

In [ ]:
df_2018 = pd.read_csv(Path("2018result.txt"))
df_2018 = df_2018[["Opp", "Result", "Starting Pitcher", "Opposing Pitcher", "Game Date"]]

true_w = df_2018["Result"].str.contains('W', na=True)
count = 0
Result = []
for row in df_2018.index:
    if true_w[row]:
        Result.append(1)
    else:
        Result.append(0)
        
df_2018["Result"] = Result
df_2018 = df_2018.set_index("Opp")

df_2018

In [ ]:
df_combined = pd.concat([df_2019,df_2018])
df_combined

## importing pitching data

In [ ]:
df_pitch_2019 = pd.read_csv(Path("mlb-player-stats-P.csv"))
df_pitch_2019.set_index(["Player"], inplace = True)
df_pitch_2019 = df_pitch_2019.drop(columns = ["Team"])
df_pitch_2019 = df_pitch_2019.groupby(level=0).mean()

df_pitch_2019.head(10)

In [ ]:
from sklearn.cluster import KMeans

inertia = []
k = list(range(1, 11))

for i in k:
    km = KMeans(n_clusters=i, random_state=5)
    km.fit(df_pitch_2019)
    inertia.append(km.inertia_)

# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

In [ ]:
model = KMeans(n_clusters=3, random_state=7)
model.fit(df_pitch_2019)
predictions = model.predict(df_pitch_2019)
df_pitch_2019["class"] = model.labels_
df_pitch_2019.head()

In [ ]:
df_pitch_2019.to_csv(Path("../Data Prep/Vu/Data/misc/test_pitch_df.csv"))

In [ ]:
df_pitch_2018 = pd.read_csv(Path("mlb-player-stats-P2018.csv"))
df_pitch_2018.set_index(["Player"], inplace = True)
df_pitch_2018 = df_pitch_2018.drop(columns = ["Team"])
df_pitch_2018 = df_pitch_2018.groupby(level=0).mean()

df_pitch_2018.head(7)

In [ ]:
inertia = []
k = list(range(1, 11))

# Looking for the best k
for i in k:
    km = KMeans(n_clusters=i, random_state=5)
    km.fit(df_pitch_2018)
    inertia.append(km.inertia_)

# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

In [ ]:
model = KMeans(n_clusters=3, random_state=7)
model.fit(df_pitch_2018)
predictions = model.predict(df_pitch_2018)
df_pitch_2018["class"] = model.labels_
df_pitch_2018.head()

In [ ]:
df_pitch_combined = pd.concat([df_pitch_2018, df_pitch_2019])

In [ ]:
inertia = []
k = list(range(1, 11))

# Looking for the best k
for i in k:
    km = KMeans(n_clusters=i, random_state=5)
    km.fit(df_pitch_combined)
    inertia.append(km.inertia_)

# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

In [ ]:
model = KMeans(n_clusters=3, random_state=7)
model.fit(df_pitch_combined)
predictions = model.predict(df_pitch_combined)
df_pitch_combined["class"] = model.labels_
df_pitch_combined.head()

In [ ]:
df_pitch_combined.to_csv(Path("df_pitch.csv"), index = True, header=True)

## importing batter data


In [ ]:
df_bat_2018 = pd.read_csv(Path("df_bat_2018.csv"))
df_bat_2019 = pd.read_csv(Path("df_bat_2019.csv"))

In [ ]:
df_bat_2019.set_index("Team",inplace=True)
df_bat_2018.set_index("Team",inplace=True)

In [ ]:
df_bat_combined = pd.concat([df_bat_2018, df_bat_2019])
df_bat_combined = df_bat_combined.groupby(level=0).mean()
df_bat_combined = df_bat_combined[["SO", "AVG", "HBP", "SH", "HR", "RBI", "R", "H"]]

df_bat_combined.head()

In [ ]:
df_hou = df_bat_combined.loc["HOU"]
df_hou = pd.DataFrame(df_hou)
df_hou = df_hou.transpose()
df_hou.rename(columns = {
    "SO" : "Astros-SO", 
    "AVG" : "Astros-AVG", 
    "HBP" : "Astros-HBP", 
    "SH" : "Astros-SH", 
    "HR" : "Astros-HR", 
    "RBI" : "Astros-RBI", 
    "R" : "Astros-R", 
    "H" : "Astros-H"
}, inplace=True)
df_hou.head()

## Home/Away Data

In [ ]:
df_home_2018 = pd.read_csv(Path("2018wl.txt"))
df_home_2019 = pd.read_csv(Path("2019wl.txt"))
df_home_2018.head()

In [ ]:
df_home_2019 = df_home_2019[["Hm/Aw", "Opp", "Game Date"]]
df_home_2019 = df_home_2019.set_index("Opp")

df_home_2018 = df_home_2018[["Hm/Aw", "Opp", "Game Date"]]
df_home_2018 = df_home_2018.set_index("Opp")


In [ ]:
df_home_away_merge = pd.concat([df_home_2019, df_home_2018])

In [ ]:
df_combined.reset_index(inplace = True)

In [ ]:
df_home_away_merge.reset_index(inplace = True)

In [ ]:
df_home_away_merge = df_home_away_merge.set_index("Game Date")
df_combined = df_combined.set_index("Game Date")

In [ ]:
merge = pd.merge(df_combined, df_home_away_merge, left_index=True, right_index=True)

merge.dropna(inplace=True)
merge

In [ ]:
merge.to_csv(Path("Results.csv"), index = True, header=True)

## combining all the dataframes

In [ ]:
merge = pd.merge(df_combined, df_bat_combined, left_index=True, right_index=True)


In [ ]:
merge["HM/AW"] =

In [ ]:
df_hou['key'] = 1
merge['key'] = 1
df = pd.merge(df_hou, merge, on='key')
del df['key']
merge = df

In [ ]:
random_number = list(range(len(merge)))

In [ ]:
opp_pitch_class = []
opp_pitch_class_2 = []
opp_pitch_class_3 = []

for number in random_number:
    if merge["Opposing Pitcher"][number] in df_pitch_combined.index:
        name = merge["Opposing Pitcher"][number]
        opp_pitch_class.append(df_pitch_combined.loc[name]["ERA"])
        opp_pitch_class_2.append(df_pitch_combined.loc[name]["WHIP"])
        opp_pitch_class_3.append(df_pitch_combined.loc[name]["K"])
    else:
        print(merge["Opposing Pitcher"][number])

In [ ]:
opp_pitch_class = []
opp_pitch_class_2 = []
opp_pitch_class_3 = []

for number in random_number:
    if merge["Opposing Pitcher"][number] in df_pitch_combined.index:
        name = merge["Opposing Pitcher"][number]
        opp_pitch_class.append(df_pitch_combined.loc[name]["class"])
    else:
        print(merge["Opposing Pitcher"][number])

In [ ]:
start_pitch_class = []
start_pitch_class_2 = []
start_pitch_class_3 = []

for number in random_number:
    if merge["Starting Pitcher"][number] in df_pitch_combined.index:
        name = merge["Starting Pitcher"][number]
        start_pitch_class.append(df_pitch_combined.loc[name]["ERA"])
        start_pitch_class_2.append(df_pitch_combined.loc[name]["WHIP"])
        start_pitch_class_3.append(df_pitch_combined.loc[name]["K"])
    else:
        print(merge["Starting Pitcher"][number])

In [ ]:
start_pitch_class = []
start_pitch_class_2 = []
start_pitch_class_3 = []

for number in random_number:
    if merge["Starting Pitcher"][number] in df_pitch_combined.index:
        name = merge["Starting Pitcher"][number]
        start_pitch_class.append(df_pitch_combined.loc[name]["class"])
    else:
        print(merge["Starting Pitcher"][number])

In [ ]:
merge["ERA-Starting "] = start_pitch_class
merge["ERA-Opposing"] = opp_pitch_class

merge["WHIP-Starting"] = start_pitch_class_2
merge["WHIP-Opposing"] = opp_pitch_class_2

merge["K-Starting"] = start_pitch_class_2
merge["K-Opposing"] = opp_pitch_class_2

In [ ]:
merge["Starting Pitcher"] = start_pitch_class
merge["Opposing Pitcher"] = opp_pitch_class

In [ ]:
merge.drop(["Starting Pitcher"], axis=1, inplace=True)
merge.drop(["Opposing Pitcher"], axis=1, inplace=True)

In [ ]:
for col in merge.columns: 
    print(col) 

## Running models

In [ ]:
X = merge.copy()
X.drop(["Result"], axis=1, inplace=True)

y = merge["Result"].ravel()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=87)

X_scaler = StandardScaler()
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Model 1 = Random Forest Classifier

In [ ]:
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

model = BalancedRandomForestClassifier(random_state=1)
model = model.fit(X_train, y_train)

y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

predictions = model.predict(X_test)
results = pd.DataFrame({"Prediction": predictions,"Actual": y_test}).reset_index(drop=True)
confusion_matrix(y_test, predictions)

print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
predictions = model.predict(X_test)
results = pd.DataFrame({"Prediction": predictions,
                        "Actual": y_test}).reset_index(drop=True)
balanced_accuracy_score(y_test, predictions)

In [ ]:
# Random Forests in sklearn will automatically calculate feature importance
importances = rf_model.feature_importances_

# We can sort the features by their importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

# Visualize the features by importance
importances_df = pd.DataFrame(sorted(zip(rf_model.feature_importances_, X.columns), reverse=True))
importances_df.set_index(importances_df[1], inplace=True)
importances_df.drop(columns=1, inplace=True)
importances_df.rename(columns={0: 'Feature Importances'}, inplace=True)
importances_sorted = importances_df.sort_values(by='Feature Importances')
importances_sorted.plot(kind='barh', color='lightgreen', title= 'Features Importances', legend=False)



## Model 2 = Decision Tree

In [ ]:
from sklearn import tree, preprocessing
from sklearn.tree import DecisionTreeClassifier

model = tree.DecisionTreeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
predictions = model.predict(X_test)
results = pd.DataFrame({"Prediction": predictions,
                        "Actual": y_test}).reset_index(drop=True)
balanced_accuracy_score(y_test, predictions)

## Model 3 - K Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

In [ ]:
knn.fit(X_train,y_train)
predictions = knn.predict(X_test)
results = pd.DataFrame({"Prediction": predictions,
                        "Actual": y_test}).reset_index(drop=True)
balanced_accuracy_score(y_test, predictions)

## Model 4 - XGBoost

In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
accuracy

In [ ]:
y_pred

In [ ]:
X_test

## Additional testing

In [ ]:
file_path = Path("mlb-player-stats-Batters.csv")
df_bat = pd.read_csv(file_path)
df_bat = df_bat.drop(columns = ["Player"])
df_bat.head()

In [ ]:
df_bat_avg = df_bat.groupby(by = "Team").mean()

In [ ]:
df_bat_avg = df_bat_avg[["SO", "AVG", "HBP", "SH", "HR", "RBI", "AVG", "R"]]

In [ ]:
df_bat_avg.head()

In [ ]:
df_new.set_index(["Opp"], inplace = True)

In [ ]:
merge = pd.merge(df_new, df_bat_avg, left_index=True, right_index=True)


In [ ]:
df.loc["Andrew Cashner"]["class"]

In [ ]:
ux = list(range(len(merge)))

In [ ]:
merge["Opposing Pitcher"][0]
# df.loc["Justin Verlander"]["class"]

new_list = []
for number in ux:
#     print(number)
#     print(merge["Opposing Pitcher"][number])
    if merge["Opposing Pitcher"][number] in df.index:
        name = merge["Opposing Pitcher"][number]
        new_list.append(df.loc[name]["class"])
#         merge["Opposing Pitcher"][number] = df.loc[name]["class"]
    else:
        print(merge["Opposing Pitcher"][number])

In [ ]:
old_list = []
for number in ux:
#     print(number)
#     print(merge["Opposing Pitcher"][number])
    if merge["Starting Pitcher"][number] in df.index:
        name = merge["Starting Pitcher"][number]
        old_list.append(df.loc[name]["class"])
#         merge["Starting Pitcher"][number] = df.loc[name]["class"]
    else:
        print(merge["Starting Pitcher"][number])

In [ ]:
merge["Starting Pitcher"] = old_list
merge["Opposing Pitcher"] = new_list

In [ ]:
merge.head()

In [ ]:
X = merge.copy()
X.drop(["Result"], axis=1, inplace=True)

y = merge["Result"].ravel()
y

In [ ]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Creating StandardScaler instance
scaler = StandardScaler()

# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=1)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)

# Calculating the confusion matrix
# cm = confusion_matrix(y_test, predictions)
# cm_df = pd.DataFrame(
#     cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
# )

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

# Random Forests in sklearn will automatically calculate feature importance
importances = rf_model.feature_importances_

# We can sort the features by their importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

# Visualize the features by importance
importances_df = pd.DataFrame(sorted(zip(rf_model.feature_importances_, X.columns), reverse=True))
importances_df.set_index(importances_df[1], inplace=True)
importances_df.drop(columns=1, inplace=True)
importances_df.rename(columns={0: 'Feature Importances'}, inplace=True)
importances_sorted = importances_df.sort_values(by='Feature Importances')
importances_sorted.plot(kind='barh', color='lightgreen', title= 'Features Importances', legend=False)

In [ ]:
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

model = BalancedRandomForestClassifier(random_state=1)
model = model.fit(X_train, y_train)

y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

predictions = model.predict(X_test)
results = pd.DataFrame({"Prediction": predictions,"Actual": y_test}).reset_index(drop=True)
confusion_matrix(y_test, predictions)

print(classification_report_imbalanced(y_test, y_pred))


In [ ]:
predictions = model.predict(X_test)
results = pd.DataFrame({"Prediction": predictions,
                        "Actual": y_test}).reset_index(drop=True)
balanced_accuracy_score(y_test, predictions)

In [ ]:
from sklearn import tree, preprocessing
from sklearn.tree import DecisionTreeClassifier

model = tree.DecisionTreeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
predictions = model.predict(X_test)
results = pd.DataFrame({"Prediction": predictions,
                        "Actual": y_test}).reset_index(drop=True)
balanced_accuracy_score(y_test, predictions)

## Model 5 - Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
predicted_y_values = model.predict(X_test)


In [ ]:
predictions = model.predict(X_test)
results = pd.DataFrame({"Prediction": predictions,
                        "Actual": y_test}).reset_index(drop=True)
balanced_accuracy_score(y_test, predictions)